In [20]:
import requests
import ftplib
import pandas as pd
import numpy as np
import tqdm
import time
from io import BytesIO
from bs4 import BeautifulSoup as bs

In [3]:
ftp_url = "oda.ft.dk"
ftp_dir = "/ODAXML/Referat/samling/"

In [4]:
dirs = []
sub_dir = []
filer = []
filer_navne = []
filer_size = []
sti_navn = []
indhold = []

r = BytesIO()


In [5]:
ftp = ftplib.FTP(ftp_url)
ftp.login("anonymous", "wpg345@alumni.ku.dk")
ftp.set_pasv(True)
ftp.cwd(ftp_dir)
ftp.dir(dirs.append)
#ftp.quit()
#dirs

In [6]:
for i in range (0, len(dirs), 1):
    sub_dir.append(ftp_dir+dirs[i][-5:]+'/') 

In [7]:
for i in range (0, len(sub_dir), 1):
    ftp.cwd(sub_dir[i])
    ftp.dir(filer.append)

In [8]:
for i in range(0, len(filer)):
    filer_navne.append(filer[i][39:])

In [9]:
for i in range(0, len(filer)):
    filer_size.append(int(filer[i][17:39]))

In [10]:
print(int(filer[0][17:38]))

2688228


In [11]:
for i in range(0, len(filer_navne)):
    sti_navn.append(ftp_dir+filer_navne[i][0:5]+'/'+filer_navne[i])

In [12]:
for i in tqdm.tqdm(range(0, len(sti_navn))):
#for i in tqdm.tqdm(range(0, 2)):
    r = BytesIO()
    ftp.retrbinary(f"RETR {sti_navn[i]}", r.write) #, blocksize=ftp_block)
    indhold.append(r.getvalue())
    r.close()
    #print(sti_navn[i])
    #time.sleep(0.75)
#print(r.getvalue())

100%|██████████████████████████████████████████████████████████████████████████████| 1489/1489 [03:57<00:00,  6.27it/s]


In [21]:
suppe = []
#for i in tqdm.tqdm(range (0,len(indhold))):
for i in tqdm.tqdm(range (0,len(indhold))):
    suppe.append(bs(indhold[i],"lxml"))

100%|██████████████████████████████████████████████████████████████████████████████| 1489/1489 [18:17<00:00,  1.36it/s]


In [24]:
df_suppe = pd.DataFrame(suppe)

C:\Users\kim\anaconda3\lib\site-packages\pandas\core\internals\construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])
